# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [29]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [30]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
#Relative path didnt work on my side, used complete
city_data_df = pd.read_csv(r"C:\Users\ulyan\UTOR-VIRT-DATA-PT-02-2024-U-LOLC\python-api-challenge\output_data\cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,tazovsky,67.4667,78.7000,-25.55,99,69,4.51,RU,1712696040
1,1,port mathurin,-19.6833,63.4167,26.81,74,0,3.24,MU,1712696040
2,2,mzimba,-11.9000,33.6000,17.32,95,14,2.62,MW,1712696041
3,3,constantia,44.1833,28.6500,10.39,78,0,2.23,RO,1712695957
4,4,jamestown,42.0970,-79.2353,23.07,43,40,5.66,US,1712695859


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [50]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 600,
    frame_height = 500,
    size = "Humidity",
    scale = 1,
    color = "City",
    alpha=0.5
)

# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [39]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_cities = city_data_df[(city_data_df["Max Temp"] >= 20) & 
                            (city_data_df["Max Temp"] <= 32) &
                            (city_data_df["Humidity"] < 60) &
                            (city_data_df["Wind Speed"] < 4) &
                            (city_data_df["Cloudiness"] < 45)]
# Drop any rows with null values
ideal_cities_nna = ideal_cities.dropna()

# Display sample data
ideal_cities_nna

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
6,6,coahuayana de hidalgo,18.7000,-103.6583,30.35,43,3,3.58,MX,1712696041
43,43,bethel,41.3712,-73.4140,24.66,34,0,2.57,US,1712696012
109,109,borazjan,29.2666,51.2159,27.17,41,0,0.18,IR,1712696051
187,187,turpan,42.9333,89.1667,21.37,17,13,3.10,CN,1712696059
201,201,laguna,38.4210,-121.4238,24.05,43,0,1.54,US,1712696052
228,228,raja,8.4596,25.6780,28.09,25,8,1.53,SS,1712696063
248,248,brisas de zicatela,15.8369,-97.0419,29.75,55,24,3.41,MX,1712696065
270,270,iranshahr,27.2025,60.6848,27.19,26,0,0.00,IR,1712696068
296,296,westport,41.1415,-73.3579,25.36,37,0,3.60,US,1712696071
317,317,selma,36.5708,-119.6121,24.02,44,0,1.54,US,1712696073


### Step 3: Create a new DataFrame called `hotel_df`.

In [40]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_cities_nna[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
6,coahuayana de hidalgo,MX,18.7000,-103.6583,43,
43,bethel,US,41.3712,-73.4140,34,
109,borazjan,IR,29.2666,51.2159,41,
187,turpan,CN,42.9333,89.1667,17,
201,laguna,US,38.4210,-121.4238,43,
228,raja,SS,8.4596,25.6780,25,
248,brisas de zicatela,MX,15.8369,-97.0419,55,
270,iranshahr,IR,27.2025,60.6848,26,
296,westport,US,41.1415,-73.3579,37,
317,selma,US,36.5708,-119.6121,44,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [41]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories": "accommodation.hotel",
    "apiKey":geoapify_key
    }

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
coahuayana de hidalgo - nearest hotel: No hotel found
bethel - nearest hotel: Hampton Inn Danbury
borazjan - nearest hotel: معلی
turpan - nearest hotel: Oriental
laguna - nearest hotel: Holiday Inn Express & Suites
raja - nearest hotel: No hotel found
brisas de zicatela - nearest hotel: Casa de Olas
iranshahr - nearest hotel: هتل قصِر
westport - nearest hotel: Norwalk Inn and Conference Center
selma - nearest hotel: Best Western
las veredas - nearest hotel: Hotel Aeropuerto
chandur - nearest hotel: No hotel found
tokar - nearest hotel: No hotel found
al jumum - nearest hotel: No hotel found
obo - nearest hotel: No hotel found
east foothills - nearest hotel: Alura Inn
barkhan - nearest hotel: No hotel found
arcadia - nearest hotel: Le Méridien Pasadena Arcadia
cambria - nearest hotel: Olallieberry Inn
newman - nearest hotel: No hotel found
palmer - nearest hotel: No hotel found
alamos - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
6,coahuayana de hidalgo,MX,18.7000,-103.6583,43,No hotel found
43,bethel,US,41.3712,-73.4140,34,Hampton Inn Danbury
109,borazjan,IR,29.2666,51.2159,41,معلی
187,turpan,CN,42.9333,89.1667,17,Oriental
201,laguna,US,38.4210,-121.4238,43,Holiday Inn Express & Suites
228,raja,SS,8.4596,25.6780,25,No hotel found
248,brisas de zicatela,MX,15.8369,-97.0419,55,Casa de Olas
270,iranshahr,IR,27.2025,60.6848,26,هتل قصِر
296,westport,US,41.1415,-73.3579,37,Norwalk Inn and Conference Center
317,selma,US,36.5708,-119.6121,44,Best Western


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [49]:
%%capture --no-display

# Configure the map plot
map_plot_hotel = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 450,
    frame_height = 500,
    size = "Humidity",
    scale = 1.5,
    color = "City",
    alpha=0.5,
    hover_cols=["Hotel Name", "Country"]
)

# Display the map
map_plot_hotel

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)